This package allows you to construct and export mtg into 4 formats : .mtg, .opf, .vtp and .gltf

In [25]:
import os
import plantconvert as pc 

ext = "opf"

#example opf filenames
simple_plant = "%s%s.%s"%("../data/", "simple_plant", ext)
coffee = "%s%s.%s"%("../data/", "coffee", ext)
palm = "%s%s.%s"%("../data/", "DA1_Average_MAP_90", ext)

Let's first read a simple plant file and visualize it with plantgl. We use an object oriented approach to implement the parser / mtg constructer because we intent to deal with complex data structure.

You can instantiate an input/output object using `format_io.io()` with the file name and then ask the object to parse the file by object method `io.read()`. The mtg will be saved in the attribute `io.g`.

The `read` method will detect the extension of the file name to determin the specific read method. You can also calling those methods without using the input/output object by calling them directly from `format_io.vtk`, `format_io.opf` and `format_io.gltf` directly.

In [26]:
import openalea.plantgl.all as pgl

In [27]:
plant = pc.plantconvert.Plant(file=simple_plant)
plant.read()

In [28]:
plant.mtg.display()

MTG : nb_vertices=7, nb_scales=4
/Individual			(id=2)											
^/Axis			(id=3)											
^/Internode			(id=4)											
	+Leaf			(id=5)										
^<Internode			(id=6)											
	+Leaf			(id=7)										


In [29]:
%gui qt
scene = pc.geometry.get_scene(plant.mtg)
pgl.Viewer.display(scene)

qt.qpa.window: <QNSWindow: 0x7fc2e428fe40; contentView=<QNSView: 0x7fc2e4324340; QCocoaWindow(0x7fc2e40ea310, window=QWidgetWindow(0x7fc2e40e9610, name="QWidgetClassWindow"))>> has active key-value observers (KVO)! These will stop working now that the window is recreated, and will result in exceptions when the observers are removed. Break in QCocoaWindow::recreateWindowIfNeeded to debug.


The meshes are stored as propreties of each node who has associated geometry, the previous function `get_scene` simply extract each geometry property and assemble them into a plantgl scene object. 

The list of reference meshes are stored as a node attribute of the node at scale 0. They are supposed to be stored in a dictionnary with mesh_id as keys and a tuple as values. The tuple contains a triangle mesh (TriangleSet object of plantgl) and a boolean that indicates if tapering should be applied on the mesh (this is made for compatibilitiy with opf and yet not very optimized...)

In [30]:
prop = plant.mtg.properties()
geo = prop['geometry']
print(geo)
ref_geo = prop["ref_meshes"]
print(ref_geo)

{4: <openalea.plantgl.scenegraph._pglsg.Shape object at 0x179766950>, 5: <openalea.plantgl.scenegraph._pglsg.Shape object at 0x1797664a0>, 6: <openalea.plantgl.scenegraph._pglsg.Shape object at 0x11aa07220>, 7: <openalea.plantgl.scenegraph._pglsg.Shape object at 0x17977e8b0>}
{0: {0: (<openalea.plantgl.scenegraph._pglsg.TriangleSet object at 0x17975f220>, True), 1: (<openalea.plantgl.scenegraph._pglsg.TriangleSet object at 0x17975f180>, True)}}


Now we can try another more complex example. To read another `opf` file, you just need to change the `file` attribute of the object `io`. And ask for parsing again using the `read` method.

In [33]:
plant.file = coffee #you can also try with palm
plant.read()

In [34]:
scene = pc.geometry.get_scene(plant.mtg)
pgl.Viewer.display(scene)

To write the currently loaded mtg into another file format, use the `write` method by providing a new filename. Let's switch back to the simple_plant as it's faster to write into file.

The [gltf](https://www.khronos.org/registry/glTF/specs/2.0/glTF-2.0.html) format is a xml styled format that allows to save a scene graph (a hierarchy of geometric objects to be rendered, we exploit this hierarchy to encode the topology of a mtg).

In [35]:
plant.file = simple_plant
plant.read()
plant.write("simple_plant.gltf")

In [36]:
# shell command to print the gltf file
%cat simple_plant.gltf

{
  "accessors": [
    {
      "bufferView": 0,
      "byteOffset": 0,
      "componentType": 5126,
      "count": 50,
      "normalized": false,
      "type": "VEC2"
    },
    {
      "bufferView": 1,
      "byteOffset": 0,
      "componentType": 5126,
      "count": 50,
      "max": [
        1.0,
        0.4990135,
        0.5
      ],
      "min": [
        0.0,
        -0.4990135,
        -0.4960575
      ],
      "normalized": false,
      "type": "VEC3"
    },
    {
      "bufferView": 1,
      "byteOffset": 600,
      "componentType": 5126,
      "count": 50,
      "normalized": false,
      "type": "VEC3"
    },
    {
      "bufferView": 2,
      "byteOffset": 0,
      "componentType": 5123,
      "count": 150,
      "normalized": false,
      "type": "SCALAR"
    },
    {
      "bufferView": 3,
      "byteOffset": 0,
      "componentType": 5126,
      "count": 12,
      "normalized": false,
      "type": "VEC2"
    },
    {
      "bufferView": 4,
      "byteOffset": 0,
     

# Write opf with geometry from a mtg file
You can load a mtg file and write it into opf format. The opf format doesn't require necessarily geometries. When you generate geometry from the mtg file and if you want to output an opf equipped with geoemtry, you should call the method `f_io.opf.writer.apply_scene(g, scene)` to associate a plantgl scene with mtg's vertices.

Inside this function, `scene.todict()` will be applied and we expect that the keys correspond to the vertices id that contains the geometry. By calling `apply_scene`, the property `geometry` and `ref_meshes` will be created and you can write opf with geometry as usual.

We intent to extract reference meshes from scene, but it's difficult to do so in general case. The method `apply_scene` saves each mesh as reference mesh and the transformation is always identity.

In `strawberry/` there is a working example using `openalea.strawberry` which provides a function to generate 3D geometry of a scene. Then we associate the geometry to the mtg and write it into opf.

>format et visu (oawidgets)

In [40]:
q = pc.plantconvert.Plant("../data/simple_plant.opf")
q.read()
scene = pc.geometry.get_scene(q.mtg)
pgl.Viewer.display(scene)

In [41]:
scene.save("simple_plant.bgeom")

In [43]:
s2 = pgl.Scene("simple_plant.bgeom")
pgl.Viewer.display(s2)